In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import linregress

In [ ]:
def compute_RS(series, min_window=10):
    """计算给定时间序列的重标极差（R/S）。"""
    n = len(series)
    RS = []
    window_sizes = range(min_window, n, int(n/4))
    for window_size in window_sizes:
        R_over_S = []
        for start in range(0, n, window_size):
            end = min(n, start + window_size)
            if end - start < window_size:
                break
            segment = series[start:end]
            mean = np.mean(segment)
            cumulative_deviation = np.cumsum(segment - mean)
            R = np.max(cumulative_deviation) - np.min(cumulative_deviation)
            S = np.std(segment)
            if S != 0:
                R_over_S.append(R / S)
        RS.append(np.mean(R_over_S))
    return np.log(window_sizes), np.log(RS)

def hurst_exponent(time_series):
    """计算时间序列的Hurst指数。"""
    lags, RS = compute_RS(time_series)
    slope, intercept, r_value, p_value, std_err = linregress(lags, RS)
    return slope

def process_csv_files(directory):
    """处理目录中的所有CSV文件，并计算它们的Hurst指数。"""
    hurst_results = {}
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            file_path = os.path.join(directory, filename)
            data = pd.read_csv(file_path)
            # 假设时间序列数据在名为'value'的列中
            time_series = data['value']
            hurst = hurst_exponent(time_series)
            hurst_results[filename] = hurst
    return hurst_results

# 使用示例
directory = 'path/to/your/csv/files'  # 将此路径替换为你的CSV文件目录
hurst_results = process_csv_files(directory)
print(hurst_results)
